In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from load_data import Data
from models import LogisticRegressor, Model, Analytics, LinearCombo
from sklearn.linear_model import LogisticRegression
import os
import pickle


In [2]:
data = Data()

pc_path = r"/media/joshua/Expansion1/tst/BaseLearnerInference"

training_images = [f"{a:03d}" for a in range(1, 33)]
all_images = [f"{a:03d}" for a in range(1, 41)]
testing_images = [f"{a:03d}" for a in range(33, 41)]

models = ["BasicPlans", "DA5_Segmentations", "LargeEncoder"]

seg_path = os.path.join(pc_path, "Probabilities")
gt_path = os.path.join(pc_path, "GroundTruths")

data.get_simple_segmentations(seg_path, all_images, ".nrrd")
data.get_groundtruths(gt_path)



In [34]:
# Get dice coefficients for each training image and model
print("Dice Coefficients by Training Image and Model:")
print("=" * 60)

for model in models:
    print(f"\n{model}:")
    model_analytics = Analytics(model_segs[model], data.gts)
    dice_scores = model_analytics.dice_coefficients()
    
    for image, dice in dice_scores.items():
        print(f"  Image {image}: {dice:.4f}")

# Or create a detailed DataFrame for training images
detailed_results = []
for model in models:
    model_analytics = Analytics(model_segs[model], data.gts)
    dice_scores = model_analytics.dice_coefficients()
    
    for image, dice in dice_scores.items():
        detailed_results.append({
            'Model': model,
            'Image': image, 
            'Dice': dice
        })

detailed_df = pd.DataFrame(detailed_results)
print("\nDetailed Training Results:")
print(detailed_df.to_string(index=False))

# Pivot table for easier comparison
pivot_df = detailed_df.pivot(index='Image', columns='Model', values='Dice')
print("\nPivot Table (Training Images vs Models):")
print(pivot_df.to_string())

Dice Coefficients by Training Image and Model:

BasicPlans:
  Image 001: 0.9733
  Image 002: 0.9666
  Image 003: 0.9726
  Image 004: 0.9803
  Image 005: 0.9532
  Image 006: 0.9850
  Image 007: 0.9659
  Image 008: 0.9787
  Image 009: 0.9723
  Image 010: 0.9780
  Image 011: 0.9763
  Image 012: 0.9789
  Image 013: 0.9855
  Image 014: 0.9818
  Image 015: 0.9565
  Image 016: 0.9735
  Image 017: 0.9617
  Image 018: 0.9020
  Image 019: 0.9761
  Image 020: 0.9773
  Image 021: 0.9791
  Image 022: 0.9639
  Image 023: 0.9620
  Image 024: 0.9696
  Image 025: 0.9666
  Image 026: 0.9645
  Image 027: 0.9205
  Image 028: 0.9694
  Image 029: 0.9519
  Image 030: 0.9700
  Image 031: 0.9713
  Image 032: 0.9748

DA5_Segmentations:
  Image 001: 0.9725
  Image 002: 0.9641
  Image 003: 0.9714
  Image 004: 0.9812
  Image 005: 0.9650
  Image 006: 0.9858
  Image 007: 0.9683
  Image 008: 0.9800
  Image 009: 0.9766
  Image 010: 0.9819
  Image 011: 0.9835
  Image 012: 0.9817
  Image 013: 0.9879
  Image 014: 0.9839


## **Base Learner Performance**

In [33]:
model_segs = {model: {} for model in models}
for model in models:
    for image in training_images:
        model_segs[model][image] = data.simple_data[image][model]



In [4]:
model_hd = {}
model_dc = {}
model_sd = {}

for model in models:
    model_analytics = Analytics(model_segs[model], data.gts)
    mean_dice = np.mean(list(model_analytics.dice_coefficients().values()))
    mean_hd = np.mean(list(model_analytics.maximum_hausdorff_distance().values()))
    model_sd[model] = np.mean(list(model_analytics.separation_distances().values()))
    model_hd[model] = mean_hd
    model_dc[model] = mean_dice



In [5]:
results_df = pd.DataFrame({
    "Model": list(model_dc.keys()),
    "Mean Dice": [model_dc[m] for m in model_dc],
    "Mean Hausdorff": [model_hd[m] for m in model_hd],
    "Mean Separation": [model_sd[m] for m in model_sd]
})

print(results_df.to_string(index=False))

            Model  Mean Dice  Mean Hausdorff  Mean Separation
       BasicPlans   0.967473        9.721207         0.580649
DA5_Segmentations   0.970390        9.057757         0.544874
     LargeEncoder   0.963385       11.351560         0.584953


## **Out Sample Performance**

In [3]:
prob_data = Data()

pc_path = r"/media/joshua/Expansion1/tst/BaseLearnerInference"

training_images = [f"{a:03d}" for a in range(1, 33)]
all_images = [f"{a:03d}" for a in range(1, 41)]
testing_images = [f"{a:03d}" for a in range(33, 41)]

models = ["BasicPlans", "DA5_Segmentations", "LargeEncoder"]

seg_path = os.path.join(pc_path, "Probabilities")
gt_path = os.path.join(pc_path, "GroundTruths")

prob_data.get_simple_segmentations(seg_path, all_images, ".npz")
prob_data.get_groundtruths(gt_path)

In [4]:
logreg = LogisticRegressor(prob_data.simple_data, prob_data.gts, models)

In [23]:
logreg.train_model(training_images, model_params = {"C": 3.1622776601683795e-5})

,penalty,'l2'
,dual,False
,tol,0.0001
,C,3.1622776601683795e-05
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [24]:
logreg_predictions = logreg.predict(testing_images)

(76, 68, 102)
(69, 66, 107)
(114, 95, 158)
(72, 86, 140)
(92, 88, 85)
(63, 58, 93)
(75, 69, 98)
(72, 75, 99)


In [25]:
analytics = Analytics(logreg_predictions, prob_data.gts)

In [26]:
model_segs = {model: {} for model in models}
for model in models:
    for image in testing_images:
        model_segs[model][image] = data.simple_data[image][model]



In [27]:
model_segs["Ensemble"] = logreg_predictions

In [28]:
model_hd = {}
model_dc = {}
model_sd = {}

for model in model_segs.keys():
    model_analytics = Analytics(model_segs[model], data.gts)
    mean_dice = np.mean(list(model_analytics.dice_coefficients().values()))
    mean_hd = np.mean(list(model_analytics.maximum_hausdorff_distance().values()))
    model_sd[model] = np.mean(list(model_analytics.separation_distances().values()))
    model_hd[model] = mean_hd
    model_dc[model] = mean_dice



In [29]:
results_df = pd.DataFrame({
    "Model": list(model_dc.keys()),
    "Mean Dice": [model_dc[m] for m in model_dc],
    "Mean Hausdorff": [model_hd[m] for m in model_hd],
    "Mean Separation": [model_sd[m] for m in model_sd]
})

print(results_df.to_string(index=False))

            Model  Mean Dice  Mean Hausdorff  Mean Separation
       BasicPlans   0.974778        5.547775         0.465259
DA5_Segmentations   0.974412        2.876370         0.476326
     LargeEncoder   0.974332        6.008247         0.470684
         Ensemble   0.975232        3.832922         0.459817


{'BasicPlans': np.float64(0.9747781948330523),
 'DA5_Segmentations': np.float64(0.9744124002546826),
 'LargeEncoder': np.float64(0.9743315525103469)}